# Simulation experiment 

Run entire simulation experiment multiple times to generate confidence interval

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../")
from functions import pipelines

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [2]:
# Parameters
NN_architecture = 'NN_2500_30'
analysis_name = 'analysis_3'
file_prefix = 'Partition_corrected'
num_simulated_experiments = 50
lst_num_partitions = [1, 2, 3, 5, 10, 20,
                    30, 50]
corrected = True
use_pca = True
num_PCs = 10

iterations = range(5) 
num_cores = 5

In [3]:
# Input files
base_dir = os.path.abspath(
  os.path.join(
      os.getcwd(), "../.."))    # base dir on repo

local_dir = local_dir = os.path.abspath(
    os.path.join(
        os.getcwd(), "../../../..")) 

normalized_data_file = os.path.join(
    local_dir,
    "Data",
    "Batch_effects",
    "input",
    "recount2_gene_normalized_data.tsv")

experiment_ids_file = os.path.join(
      base_dir,
      "data",
      "metadata",
      "recount2_experiment_ids.txt")

In [4]:
# Output files
local_dir = "/home/alexandra/Documents/"

similarity_corrected_file = os.path.join(
    local_dir,
    "Data",
    "Batch_effects",
    "output",
    "analysis_3_similarity_corrected.pickle")

ci_corrected_file = os.path.join(
    local_dir,
    "Data",
    "Batch_effects",
    "output",
    "analysis_3_ci_corrected.pickle")

In [5]:
# Run multiple simulations - corrected
results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.matched_simulation_experiment_corrected)(i,
                                                           NN_architecture,
                                                           analysis_name,
                                                           num_simulated_experiments,
                                                           lst_num_partitions,
                                                           corrected,
                                                           use_pca,
                                                           num_PCs,
                                                           file_prefix,
                                                           normalized_data_file,
                                                           experiment_ids_file) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed: 107.8min
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 117.5min remaining: 176.2min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 135.9min remaining: 90.6min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 168.8min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 168.8min finished


In [6]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score
number of partitions,,,,,
1,0.999949,0.999970,0.999964,0.999954,0.999948
2,0.979882,0.975291,0.983525,0.973538,0.977819
3,0.966441,0.960275,0.954413,0.943619,0.978614
5,0.934006,0.911218,0.934027,0.936373,0.927504
10,0.840406,0.775607,0.865168,0.829757,0.832433
20,0.761875,0.704679,0.766053,0.726482,0.737391
30,0.646205,0.578022,0.622732,0.549295,0.610607
50,0.326248,0.374420,0.403897,0.373728,0.384665


In [7]:
# Get median for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of partitions,
1,0.999957
2,0.978011
3,0.960672
5,0.928626
10,0.828674
20,0.739296
30,0.601372
50,0.372592


In [8]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of partitions,
1,0.000003
2,0.001238
3,0.004139
5,0.003250
10,0.010369
20,0.008050
30,0.012051
50,0.009053


In [9]:
# Get confidence interval for each row (number of experiments)
err = std_scores*1.96

In [10]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of partitions,,
1,0.999951,0.999963
2,0.975584,0.980438
3,0.952559,0.968785
5,0.922256,0.934995
10,0.808350,0.848998
20,0.723519,0.755074
30,0.577753,0.624991
50,0.354848,0.390335


In [11]:
mean_scores

,score
number of partitions,
1,0.999957
2,0.978011
3,0.960672
5,0.928626
10,0.828674
20,0.739296
30,0.601372
50,0.372592


In [12]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_corrected_file)
ci.to_pickle(ci_corrected_file)